<a href="https://colab.research.google.com/github/MarkNgendo/Bluecar-electric-car-services/blob/main/Moringa_Data_Science_Prep_W4_Independent_Project_2021_03_Mark_Ngendo_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WEEK 4 IP: PARIS BLUECAR PROJECT

## 1.1 Research Objective

In this week's independent project, you will be working as a data scientist working for an electric car-sharing service company.

**Research Question**

Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.

Some further questions that will be answered in this Notebook will be:

* What is the most popular hour for returning cars?
* What station is the most popular?
  * Overall?
  * At the most popular picking hour?
* What postal code is the most popular for picking up Blue  cars? Does the most popular station belong to that postalcode?
  * Overall?
  * At the most popular picking hour?
* Do the results change if you consider Utilib and Utilib 1.4 instead of Blue cars? 

## 1.2 Importing libraries


In [50]:
import pandas as pd
import numpy as np

## 1.3 Importing the data set.

The datset and its descriptions can be gotten from the following respective links:
* Data set : http://bit.ly/autolib_dataset
* Desciption : https://drive.google.com/file/d/13DXF2CFWQLeYxxHFekng8HJnH_jtbfpN/view

In [51]:
df = pd.read_csv('Autolib_dataset (2).csv')

### 1.3.1 Previewing the dataset

In [52]:
df.head()

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23


### 1.3.2 Accessing more information on the dataset

This helps in knowing the format of the values within the columns.

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Geo point            5000 non-null   object
 12  Postal code          5000 non-null   int64 
 13  Public name          5000 non-null   object
 14  Rental status        5000 non-null   object
 15  Scheduled at         47 non-null     object
 16  Slots 

## 1.4 Cleaning the data set

Before any analysis can be performed, the dataset needs to undergo sufficient cleaning to make the analysis as efficient and accurate as possible.

### 1.4.1 Validity

In [54]:
# Dropping irrelevant columns that have no bearing in the analysis

df = df.drop(['Cars', 'Displayed comment', 'Geo point', 'ID', 'Scheduled at',
          'Station type', 'Subscription status', 'Charge Slots', 'Status', 'Charging Status', 'year','month', 'Address'], axis=1)
df.head()

,Bluecar counter,Utilib counter,Utilib 1.4 counter,City,Kind,Postal code,Public name,Rental status,Slots,day,hour,minute
0,0,0,0,Paris,STATION,75015,Paris/Suffren/2,operational,2,8,11,43
1,6,0,0,Paris,STATION,75014,Paris/Raymond Losserand/145,operational,0,6,7,24
2,3,0,2,Le Bourget,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,3,20,14
3,3,1,0,Paris,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,4,4,37
4,3,0,0,Paris,PARKING,75017,Paris/Porte de Champerret/6,operational,3,8,17,23


### 1.4.2 Completeness

In [55]:
# checking for null values

df.isnull().sum()

Bluecar counter       0
Utilib counter        0
Utilib 1.4 counter    0
City                  0
Kind                  0
Postal code           0
Public name           0
Rental status         0
Slots                 0
day                   0
hour                  0
minute                0
dtype: int64

### 1.4.3 Consistency

In [56]:
# checking for duplicates

df.duplicated().sum()

0

### 1.4.4 Uniformity

In [57]:
df.columns = df.columns.str.lower()
df.head()

,bluecar counter,utilib counter,utilib 1.4 counter,city,kind,postal code,public name,rental status,slots,day,hour,minute
0,0,0,0,Paris,STATION,75015,Paris/Suffren/2,operational,2,8,11,43
1,6,0,0,Paris,STATION,75014,Paris/Raymond Losserand/145,operational,0,6,7,24
2,3,0,2,Le Bourget,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,3,20,14
3,3,1,0,Paris,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,4,4,37
4,3,0,0,Paris,PARKING,75017,Paris/Porte de Champerret/6,operational,3,8,17,23


## 1.5 Exporting the cleaned dataset

This is useful for future reference.

In [58]:
df.to_csv('cars_cleaned.csv')

## 1.5 Analysis

#### Most popular pick up hour in Paris

In [98]:
#Loading the cleaned set now
df = pd.read_csv('cars_cleaned.csv')
df.head()

,Unnamed: 0,bluecar counter,utilib counter,utilib 1.4 counter,city,kind,postal code,public name,rental status,slots,day,hour,minute
0,0,0,0,0,Paris,STATION,75015,Paris/Suffren/2,operational,2,8,11,43
1,1,6,0,0,Paris,STATION,75014,Paris/Raymond Losserand/145,operational,0,6,7,24
2,2,3,0,2,Le Bourget,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,3,20,14
3,3,3,1,0,Paris,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,4,4,37
4,4,3,0,0,Paris,PARKING,75017,Paris/Porte de Champerret/6,operational,3,8,17,23


In [99]:
# Creating difference columns.
df['blusage'] = df['bluecar counter'].diff()
df['usage'] = df['utilib counter'].diff()
df['u4usage'] = df['utilib 1.4 counter'].diff()

In [100]:
# Before answering the research questions, the data has to be grouped efficiently
# Also, since we are mainly interested with the city of Paris, we need to filter 
# the dataset out.
 

Paris = df[(df['city']=='Paris') & (df['blusage'] < 0)]
Paris.groupby(['hour'])[['blusage']].sum().sort_values(by='blusage', ascending=True).head()



,blusage
hour,
21,-231.0
7,-196.0
20,-193.0
5,-190.0
16,-172.0


From this result, it is easily identifiable which hour in Paris was the most popular for Bluecar pick ups:


Hour 21(9 pm) had 231 cars being rented out across Paris.

#### Determining the most popular station

(All cities included)

In [115]:
df4 = df[(df['kind'] == 'STATION') & (df['rental status'] == 'operational')]

df4stat = df4.groupby(['public name'])[['public name']].count()
df4stat.columns = ['Count']
df4stat

,Count
public name,
Alfortville/Charles de Gaulle/16,5
Alfortville/Etienne Dolet/174,2
Alfortville/Jean-Baptiste Preux/17,4
Alfortville/Joseph Franceschi/1,5
Alfortville/Port Ã l'Anglais/39,7
...,...
Yerres/Pierre Brossolette/92,5
Yerres/Raymond PoincarÃ©/200,1
Ãpinay-sur-Seine/Avenir/1,4


In [116]:
# sorting them by count to determine the most popular

df4stat_sort = df4stat.sort_values(by='Count', ascending=False)
df4stat_sort.head()

,Count
public name,
Paris/Porte de Montrouge/8,13
Paris/Voltaire/182,11
SÃ¨vres/WolfenbÃ¼ttel/1,11
Paris/Philippe Auguste/126,11
Paris/Mathis/35,11


From this, we can see that the most popular station is
* Paris/Porte de Montrouge/8	
 count = 13
 
 This is Overall.


In [103]:
# During the most popular pick up time, 1400hrs, the most popular station should
# be

hour21 = df[df['hour'] == 21]
h21 = hour21.groupby(['public name'])[['public name']].count()
h21.columns = ['count']
h21 = h21.sort_values(by='count', ascending = False)
h21.head()

,count
public name,
Paris/Tronchet/19,4
Paris/Voltaire/182,3
Alfortville/Europe/22,2
Paris/HÃ´pital/47,2
Paris/Picpus/122,2


From this, the most Popular station during peak pick up hours is:
* Paris/Tronchet/19
by count : 4

#### Postal code for most popular pick up station (Bluecar)

In [104]:
# Overall
postal = df4.groupby(['postal code'])[['postal code']].count()
postal.columns = ['count']
postal = postal.sort_values(by='count', ascending=False)
postal.head()

,count
postal code,
75015,220
75016,197
75017,180
75011,155
75013,155


In [105]:
df4[df4['public name'] == 'Paris/Tronchet/19'][['public name', 'postal code']].head(1)

,public name,postal code
600,Paris/Tronchet/19,75008


The most popular postal code overall is 75015.
The most popular station overall does not belong to this postal code.

In [106]:
# During the most popular hour:

postal = hour21.groupby(['postal code'])[['postal code']].count()
postal.columns = ['count']
postal = postal.sort_values(by='count', ascending=False)
postal.head()


,count
postal code,
75014,16
75019,15
75016,13
75012,12
75015,11


In [107]:
df4[df4['public name'] == 'Paris/Tronchet/19'][['public name', 'postal code']].head(1)

,public name,postal code
600,Paris/Tronchet/19,75008


The most popular postal code is still 75014 and the most popular station at that hour still does not belong to it.

#### Most popular hour to return Bluecar cars

In [161]:
c = df[df['blusage'] > 0]
c.groupby(['hour'])[['blusage']].sum().sort_values(by='blusage', ascending=False).head()

,blusage
hour,
6,313.0
5,287.0
12,279.0
2,276.0
3,269.0


The most popular hour to return Bluecar cars was 6 AM

### 1.5.2   Analysis for utilib and utilib 1.4

#### Utilib cars

##### Most popular hourd for returning Utilib Cars

In [108]:
back = df4.groupby(['public name','hour'])[['utilib counter']].sum()
back['urented'] = back['utilib counter'].diff()
back = back[back['urented'] > 0]
back.head()

,,utilib counter,urented
public name,hour,,
Alfortville/Jean-Baptiste Preux/17,23,1,1.0
Alfortville/Port Ã l'Anglais/39,9,1,1.0
Antony/Anciens combattants d'Afrique du Nord/1,16,1,1.0
Arcueil/Marcel Vigneron/30,1,1,1.0
Argenteuil/Poste ProlongÃ©e/1,18,1,1.0


In [109]:
backu = back.groupby(['hour'])[['urented']].sum().sort_values(by='urented', ascending=False)
backu.head()

,urented
hour,
7,16.0
8,15.0
5,14.0
2,13.0
6,13.0


7 AM is the prime hour for returning Utilib vehicles.

##### Most popular rental hours for Utilib

In [138]:
util  = df[df['usage'] < 0]
util.groupby(['hour'])[['usage']].sum().sort_values(by='usage').head()

,usage
hour,
11,-17.0
6,-17.0
14,-17.0
10,-16.0
2,-15.0


The most popular hour is 1100 hours.
However,the most popular station in general does not change.

##### Most popular station at peak renting hours

In [156]:
# Most popular utuilib station at most popular hours.
df2u = df4[df4['hour']==11]

df2usort = df2u.groupby(['public name'])[['public name']].count()
df2usort.columns = ['count']
df2usort = df2usort.sort_values(by='count', ascending = False)
df2usort.head(10)


,count
public name,
Paris/Saint-HonorÃ©/123,2
Saint-Ouen/Capitaine Glarner/44,2
Paris/Pereire/170,2
Nanterre/Waldeck Rochet/8,2
Nanterre/Lenine/65,2
Paris/Ãmile Augier/20,2
Clamart/Ferrari/1,2
Saint-MandÃ©/Pasteur/11,2
Paris/Ivry/3,2


From the above result, the first 9 stations hold the title for most popular at this time.

##### Most popular postal code for Utilib

In [147]:
# Overall popular postal code for utilib cars
upostal = df4.groupby(['postal code'])[['postal code']].count()
upostal.columns = ['count']
upostal = upostal.sort_values(by='count', ascending=False)
upostal.head()

,count
postal code,
75015,220
75016,197
75017,180
75011,155
75013,155


Most popular station at Peak hours

In [163]:
upostal = df2u.groupby(['postal code'])[['postal code']].count()
upostal.columns = ['count']
upostal = upostal.sort_values(by='count', ascending=False)
upostal.head()

,count
postal code,
75017,15
75013,11
75015,9
92000,8
75018,7


##### Matching postal code and popular station

In [158]:
# Does this postal code belong to either of the listed public names?

a = list(df2usort.index)
a

for x in a[:9]:
  for y in list(df4[df4['public name'] ==x]['postal code']):
     if y == str(75015):
      print(x, 75015)

# no match found

None of the above listed stations belong to the most popular postal code in peak renting time (1900 hours)

#### Utilib 1.4 cars


##### Most popular time for returning Utilib 1.4 cars

In [119]:
ret = df[df['u4usage'] > 0]
ret.groupby(['hour'])[['u4usage']].sum().sort_values(by='u4usage', ascending=False).head()

,u4usage
hour,
3,32.0
5,29.0
14,28.0
6,28.0
9,28.0


The most popular time for returning Utilib 1.4 cars is 0300 hours.

##### Most popular hour for renting utilib 1.4 hours

In [121]:
ut14 = df[df['u4usage'] < 0]
ut14.groupby(['hour'])[['u4usage']].sum().sort_values(by='u4usage', ascending=True).head()

,u4usage
hour,
20,-28.0
4,-28.0
11,-27.0
3,-27.0
21,-26.0


##### Most popular station for the 1.4 cars.

The general popularity does not change

In [159]:
df2zhour = df4[df4['hour']==20]
df2zsort = df2zhour.groupby(['public name'])[['public name']].count()
df2zsort.columns = ['count']
df2zsort = df2zsort.sort_values(by='count', ascending = False)
df2zsort.head()

,count
public name,
Paris/Paradis/28,4
Paris/RapÃ©e/46,3
Paris/Astorg/11,3
Paris/PerrÃ©e/18,2
Romainville/Commune de Paris/100,2


##### Most popular postal code for utiib 1.4 cars

In [136]:
df2zpostal = df4.groupby(['postal code'])[['postal code']].count()
df2zpostal.columns = ['count']
df2zpostal = df2zpostal.sort_values(by='count', ascending = False)
df2zpostal.head()


,count
postal code,
75015,220
75016,197
75017,180
75011,155
75013,155


The order remains the same just as like the other cars.

Most popular station at Peak hours

In [164]:
df2zpostal = df2zhour.groupby(['postal code'])[['postal code']].count()
df2zpostal.columns = ['count']
df2zpostal = df2zpostal.sort_values(by='count', ascending = False)
df2zpostal.head()

,count
postal code,
75010,9
75015,8
94000,8
75020,7
75018,7


##### Matching postal code to popular stations.




In [160]:
stations = list(df2zsort.index)

for x in stations[0]:
  for y in list(df4[df4['public name'] ==x]['postal code']):
     if y == str(75015):
      print(x, 75015)

# no match